In [1]:
import glob
import gzip
import logging
import os
import shutil

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray

logging.basicConfig(
    format="%(asctime)s - %(message)s", level=logging.INFO
)  # before any glmtools are imported. it starts a logger to who knows?
import glm as myglm
from glmtools.io.glm import GLMDataset
from glmtools.plot.locations import plot_flash
from grid.make_GLM_grids import create_parser, grid_setup

In [2]:
parser = create_parser()
args = parser.parse_args([])

startdate, lon_range, lat_range = (
    pd.to_datetime("20210710T08"),
    (-101, -87),
    (37, 43),
)  # upper midwest
startdate, lon_range, lat_range = (
    pd.to_datetime("20180516T02"),
    (-103, -96),
    (29, 36),
)  # waf-d-19-0141.1 Fig 3a) Mallard fire
# Find flashes in some location.

dx = 81.2705
dx = 10.0

In [3]:
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(16, 9), subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), linewidth=0.25)
    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), linewidth=0.25)
    ax.add_feature(cfeature.STATES.with_scale("50m"), linewidth=0.15)
    ax.add_feature(
        cfeature.LAKES.with_scale("50m"),
        edgecolor="k",
        linewidth=0.25,
        facecolor="k",
        alpha=0.05,
    )
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    return fig, ax


def glm_files(args, kwargs):
    xdeg = f"{kwargs['dx'][0]:.4f}deg-dx_"
    logging.debug(args)
    start = pd.to_datetime(args.start).strftime("%Y%m%d_%H%M%S")
    search_str = os.path.join(
        args.outdir, f'{kwargs["output_filename_prefix"]}_{start}_{args.dt:.0f}_*{xdeg}*.nc'
    )
    ifiles = glob.glob(search_str)
    logging.info(f"found {len(ifiles)} files matching {search_str}")
    return ifiles

In [4]:
duration = pd.to_timedelta("1H")
enddate = startdate + duration
filenames = myglm.download(startdate, enddate, bucket="noaa-goes16")

2024-10-16 10:29:16,079 - fs.ls(noaa-goes16/GLM-L2-LCFA/2018/136/02)
2024-10-16 10:29:16,494 - fs.ls(noaa-goes16/GLM-L2-LCFA/2018/136/03)


In [5]:
(width,) = np.diff(lon_range) * 111.1 * np.cos(np.radians(np.array(lat_range).mean()))
(height,) = np.diff(lat_range) * 111.1

args.lcc = False
args.x_bnd = lon_range
args.y_bnd = lat_range
args.ctr_lat = np.array(lat_range).mean()
args.ctr_lon = np.array(lon_range).mean()
args.spatial_scale_factor = 1
args.width = width
args.height = height
args.dx = dx
args.dy = dx
args.dt = duration.total_seconds()
args.filenames = filenames
args.goes_position = "east"
args.ngroups = 1
args.outdir = os.path.join(os.getenv("TMPDIR"), "GLM")
args.start = startdate.isoformat()
args.end = enddate.isoformat()
args

Namespace(filenames=['/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360200000_e20181360200200_c20181360200225.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360200200_e20181360200400_c20181360200423.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360200400_e20181360201000_c20181360201024.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360201000_e20181360201200_c20181360201226.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360201200_e20181360201400_c20181360201424.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360201400_e20181360202000_c20181360202027.nc', '/glade/derecho/scratch/ahijevyc/noaa-goes16/GLM-L2-LCFA/2018/136/02/OR_GLM-L2-LCFA_G16_s20181360202000_e20181360202200_c201813602022

## Gridded lightning density

In [6]:
gridder, glm_filenames, start_time, end_time, grid_kwargs = grid_setup(args)
grid_kwargs.update({"output_filename_prefix": "G16"})

nc_files = glm_files(args, grid_kwargs)

if len(nc_files) == 0:
    _ = gridder(glm_filenames, start_time, end_time, **grid_kwargs)
    nc_files = glm_files(args, grid_kwargs)

grd = xarray.open_mfdataset(nc_files)
grd = grd.rename(ntimes="time")
grd = grd.assign_coords(
    dict(lon=grd.longitude, lat=grd.latitude, time=[start_time + duration / 2])
)
grd

AttributeError: 'str' object has no attribute 'floor'

In [ ]:
nc_files

In [ ]:
f = grd["group_extent_density"]
f = f.where(f > 0)
f

In [ ]:
# Set the axes using the specified map projection
map_crs = ccrs.LambertConformal(central_latitude=35, central_longitude=-100)
map_crs = ccrs.PlateCarree()
fig, ax = make_map(bbox=[*lon_range, *lat_range], projection=map_crs)
f.plot(x="lon", ax=ax, transform=ccrs.PlateCarree(), cmap="tab20c_r", alpha=0.8)

## MRMS

In [ ]:
idir = "/glade/campaign/mmm/parc/sobash/MRMS"  # hourly files
glmt = start_time + duration / 2
search_str = f"{idir}/{glmt.strftime('%Y%m%d')}"
search_str += "/MRMS_MergedReflectivityQCComposite_00.50_"
search_str += f"{glmt.strftime('%Y%m%d-%H')}*.grib2.gz"
logging.info(f"MRMS search_str={search_str}")
(gzfile,) = glob.glob(search_str)
logging.info(f"closest mrms {gzfile}")

base, ext = os.path.splitext(gzfile)
targetdir = os.getenv("TMPDIR")
grb = os.path.join(targetdir, os.path.basename(base))  # without .gz

if not os.path.exists(grb):
    logging.info(f"gunzip {gzfile}")
    with gzip.open(gzfile, "rb") as f_in:
        logging.info(f"write {grb}")
        with open(grb, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

logging.info(f"opening dataset {grb}")
ds = xarray.open_dataset(
    grb, engine="cfgrib", backend_kwargs={"indexpath": "{path}.{short_hash}.idx"}
)
ds

In [ ]:
dbz = ds.unknown
logging.debug("reindexing lon")
dbz = dbz.assign_coords(longitude=((dbz.longitude + 180) % 360) - 180)
logging.debug("flipping lat")
dbz = dbz.reindex(latitude=dbz.latitude[::-1])
dbz = dbz.where(dbz > -30)
slice_dbz = dbz.sel(longitude=slice(*lon_range), latitude=slice(*lat_range))
fig, ax = make_map(bbox=[*lon_range, *lat_range], projection=ccrs.PlateCarree())
slice_dbz.plot(
    ax=ax,
    x="longitude",
    y="latitude",
    cmap="tab20c_r",
    transform=ccrs.PlateCarree(),
    vmin=-5,
    vmax=80,
)

In [ ]:
def do_plot(dbz, lon_range, lat_range, glmfiles=[], grid=None):
    ax.imshow(
        dbz,
        extent=(*lon_range, *lat_range),
        origin="lower",
        cmap="tab20c_r",
        transform=ccrs.PlateCarree(),
    )
    for f in glmfiles:
        glm = GLMDataset(f)
        flashes_subset = glm.subset_flashes(lon_range=lon_range, lat_range=lat_range)
        for flash_id in flashes_subset.flash_id:
            _ = plot_flash(glm, flash_id, ax, ccrs.PlateCarree())
    if grid is not None:
        grid.isel(time=0).plot.pcolormesh(
            x="lon", y="lat", ax=ax, cmap="Set1_r", alpha=0.7, transform=ccrs.PlateCarree()
        )

In [ ]:
filenames = myglm.download(startdate, enddate, bucket="noaa-goes16")
fig, ax = make_map(bbox=[*lon_range, *lat_range], projection=map_crs)
do_plot(slice_dbz, lon_range, lat_range, grid=f)

In [ ]:
fig, ax = make_map(bbox=[*lon_range, *lat_range], projection=map_crs)
do_plot(slice_dbz, lon_range, lat_range, glmfiles=filenames)

![fig](https://journals.ametsoc.org/view/journals/wefo/35/2/full-waf-d-19-0141.1-f3.jpg)

In [ ]:
filenames = myglm.download(startdate, enddate, bucket="noaa-goes17")
args.filenames = filenames
gridder, glm_filenames, start_time, end_time, grid_kwargs = grid_setup(args)

grid_kwargs.update({"output_filename_prefix": "G17"})

nc_files = glm_files(args, grid_kwargs)

if len(nc_files) == 0:
    _ = gridder(glm_filenames, start_time, end_time, **grid_kwargs)
    nc_files = glm_files(args, grid_kwargs)

grd = xarray.open_mfdataset(nc_files)
grd = grd.rename(ntimes="time")
grd = grd.assign_coords(
    dict(lon=grd.longitude, lat=grd.latitude, time=[start_time + duration / 2])
)
grd = grd["flash_extent_density"]
grd = f.where(grd > 0)
fig, ax = make_map(bbox=[*lon_range, *lat_range], projection=map_crs)
do_plot(slice_dbz, lon_range, lat_range, glmfiles=filenames, grid=grd)

In [ ]:
glm = GLMDataset(filenames[0])
from glmtools.io.mimic_lma import read_flashes

d = read_flashes(glm, target=None, clip_events=False)
d[0]

In [ ]:
d[0]["flash"][1]